In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer,f1_score, accuracy_score, precision_score
from sklearn.model_selection import GridSearchCV

#Text pre-processing
"""removes punctuation, stopwords, and returns a list of the remaining words, or tokens"""
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv('all_tickets.csv')

In [3]:
newdf = df.select_dtypes([np.number])
newdf.columns.values

array(['ticket_type', 'category', 'sub_category1', 'sub_category2',
       'business_service', 'urgency', 'impact'], dtype=object)

In [4]:
#User defined function

def content_extractor(content, start=None, end=None):

  try:

    if start and content and end:

      builder="{}(.*)(?={})".format(start,end)

      pattern=re.compile(builder)

      return pattern.search(content).group(0)

    else:

      return content

  except Exception as e:

    return content

    

ser1=df.apply(lambda x: content_extractor(x,"start_text","end_text"))

In [5]:
y= pd.DataFrame(df['ticket_type'])
X= df.drop(columns=["title","ticket_type"])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X['body'], y, test_size=0.4, random_state=0)
X_train

27120    tuesday november pm maternity leaver form hell...
38261    error hello please be aware had recovery error...
37829    error when attempting access hi please escalat...
31054    tuesday needed today hi today presentation aft...
15808    pm addresses hi please addresses corresponding...
                               ...                        
21243    hi created order add user accounts created use...
45891    sent thursday client portal environment open p...
42613    sent tuesday update possible dear colleagues s...
43567    sent friday issue hello have problem with bein...
2732     hands event hi help setting technical details ...
Name: body, Length: 29129, dtype: object

In [7]:
#Cleaning the text

import string
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Return the cleaned text as a list of words
    4. Remove words
    '''
    stemmer = WordNetLemmatizer()
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join([i for i in nopunc if not i.isdigit()])
    nopunc =  [word.lower() for word in nopunc.split() if word not in stopwords.words('english')]
    return [stemmer.lemmatize(word) for word in nopunc]

sample_text = "Hey There! This is a Sample review, which 123happens {blah}%456 to contain happened punctuations universal rights of right contained."
print(text_process(sample_text))

#Vectorisation : -

from sklearn.feature_extraction.text import TfidfVectorizer

tfidfconvert = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer=text_process,ngram_range=(1, 3)).fit(X_train)
print("tfidfconvert----------------",tfidfconvert)
X_transformed=tfidfconvert.transform(X_train)
print("X_transformed----------------",X_transformed)



['hey', 'there', 'this', 'sample', 'review', 'happens', 'blah', 'contain', 'happened', 'punctuation', 'universal', 'right', 'right', 'contained']


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:498: UserWarning: The parameter 'ngram_range' will not be used since 'analyzer' is callable'
  warnings.warn("The parameter 'ngram_range' will not be used"


tfidfconvert---------------- TfidfVectorizer(analyzer=<function text_process at 0x00000233D5C361F0>,
                ngram_range=(1, 3), norm=None, smooth_idf=False)
X_transformed----------------   (0, 8507)	2.783724276491034
  (0, 8235)	1.9106934628254955
  (0, 7785)	4.575075165529544
  (0, 5910)	1.9502112244459544
  (0, 5900)	1.2957592802813203
  (0, 5348)	3.607197064864896
  (0, 4896)	11.124923638174858
  (0, 4551)	4.926860124174084
  (0, 3742)	2.655057578407815
  (0, 3741)	2.001116738125641
  (0, 3331)	7.217189378263067
  (0, 3187)	6.114703546570137
  (0, 3036)	3.182672049921332
  (1, 8235)	1.9106934628254955
  (1, 6531)	6.2755432145481915
  (1, 5900)	1.2957592802813203
  (1, 5717)	6.45117578319135
  (1, 3741)	2.001116738125641
  (1, 3591)	5.2018472771446165
  (1, 3235)	5.55917774388624
  (1, 2837)	6.729743622905944
  (1, 655)	6.8850403658212125
  (1, 59)	4.355860892355224
  (2, 8224)	4.378758856448478
  (2, 7818)	4.93560908636732
  :	:
  (29127, 412)	4.0745970102889775
  (29127, 3

In [8]:
#transform the test features to sparse matrix
test_features = tfidfconvert.transform(X_test)

In [9]:
test_features

<19420x9189 sparse matrix of type '<class 'numpy.float64'>'
	with 467564 stored elements in Compressed Sparse Row format>

In [10]:
# Clustering the training sentences with K-means technique

from sklearn.cluster import KMeans
modelkmeans = KMeans(n_clusters=12, init='k-means++', n_init=100)
modelkmeans.fit(X_transformed)

KMeans(n_clusters=12, n_init=100)

In [18]:
y_hat = modelkmeans.predict(X_transformed)

In [19]:
y_hat

array([10, 10, 10, ..., 10, 10, 10])